In [1]:
"""
This simple notebook demonstrates the workflow of using the TensorFlow converter.
"""
import numpy as np
from tensorflow.python.tools.freeze_graph import freeze_graph

import tfcoreml
import linear_mnist_train

In [2]:
"""
Step 0: Before you run this notebook, view run the example script linear_mnist_train.py
to get a trained TensorFlow network.
This may take a few minutes.
"""
linear_mnist_train.train()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
image_input:  Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
pred output: Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Epoch: 0001 cost= 1.183501422
Epoch: 0002 cost= 0.665199664
Epoch: 0003 cost= 0.552743921
Epoch: 0004 cost= 0.498638443
Epoch: 0005 cost= 0.465489734
Epoch: 0006 cost= 0.442558188
Epoch: 0007 cost= 0.425494437
Epoch: 0008 cost= 0.412193335
Epoch: 0009 cost= 0.401347567
Epoch: 0010 cost= 0.392438287
Epoch: 0011 cost= 0.384738888
Epoch: 0012 cost= 0.378189654
Epoch: 0013 cost= 0.372379461
Epoch: 0014 cost= 0.367302661
Epoch: 0015 cost= 0.362724612
Epoch: 0016 cost= 0.358586604
Epoch: 0017 cost= 0.354892586
Epoch: 0018 cost= 0.351457737
Epoch: 0019 cost= 0.348290575
Epoch: 0020 cost= 0.345420393
Epoch: 0021 cost= 0.342759541
Epoch: 0022 cost= 0.340266625
Epoch: 0023 cost= 

In [3]:
"""
Step 1: "Freeze" your tensorflow model - convert your TF model into a stand-alone graph definition file
Inputs: 
(1) TensorFlow code
(2) trained weights in a checkpoint file
(3) The output tensors' name you want to use in inference
(4) [Optional] Input tensors' name to TF model
Outputs: 
(1) A frozen TensorFlow GraphDef, with trained weights frozen into it
"""

# Provide these to run freeze_graph:
# Graph definition file, stored as protobuf TEXT
graph_def_file = './model.pbtxt'
# Trained model's checkpoint name
checkpoint_file = './checkpoints/model.ckpt'
# Frozen model's output name
frozen_model_file = './frozen_model.pb'
# Output nodes. If there're multiple output ops, use comma separated string, e.g. "out1,out2".
output_node_names = 'Softmax' 


# Call freeze graph
freeze_graph(input_graph=graph_def_file,
             input_saver="",
             input_binary=False,
             input_checkpoint=checkpoint_file,
             output_node_names=output_node_names,
             restore_op_name="save/restore_all",
             filename_tensor_name="save/Const:0",
             output_graph=frozen_model_file,
             clear_devices=True,
             initializer_nodes="")


INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt


INFO:tensorflow:Restoring parameters from ./checkpoints/model.ckpt


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.


In [4]:
"""
Step 2: Call converter
"""

# Provide these inputs in addition to inputs in Step 1
# A dictionary of input tensors' name and shape (with batch)
input_tensor_shapes = {"Placeholder:0":[1,784]} # batch size is 1
# Output CoreML model path
coreml_model_file = './model.mlmodel'
output_tensor_names = ['Softmax:0']


# Call the converter
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file, 
        mlmodel_path=coreml_model_file, 
        input_name_shape_dict=input_tensor_shapes,
        output_feature_names=output_tensor_names)


Shapes not found for 4 tensors. Executing graph to determine shapes. 
2/8: Converting op name: Variable_1/read ( type:  Identity )
4/8: Converting op name: Variable/read ( type:  Identity )
5/8: Converting op name: Placeholder ( type:  Placeholder )
Skipping name of placeholder
6/8: Converting op name: MatMul ( type:  MatMul )
8/8: Converting op name: Softmax ( type:  Softmax )

 Core ML model generated. Saved at location: ./model.mlmodel 

Core ML input(s): 
 [name: "Placeholder__0"
type {
  multiArrayType {
    shape: 784
    dataType: DOUBLE
  }
}
]
Core ML output(s): 
 [name: "Softmax__0"
type {
  multiArrayType {
    shape: 10
    dataType: DOUBLE
  }
}
]


In [5]:
"""
Step 3: Run the converted model
"""

# Provide CoreML model with a dictionary as input. Change ':0' to '__0'
# as Swift / Objective-C code generation do not allow colons in variable names
coreml_inputs = {'Placeholder__0': np.random.rand(1,1,784)} # (sequence_length=1,batch=1,channels=784)
coreml_output = coreml_model.predict(coreml_inputs, useCPUOnly=False)


In [6]:
print coreml_output

{u'Softmax__0': array([  2.36593969e-02,   9.70071051e-06,   2.34215289e-01,
         1.44295856e-01,   1.23145210e-03,   4.75908250e-01,
         1.82468928e-02,   6.08514296e-03,   9.40786675e-02,
         2.26941495e-03])}
